# EDA

Study of a dataset cotaining various information about 32135 games published on Steam gaming plataform.

| Columna | Descripción | Ejemplo |
| -- | - | - |
| publisher | Empresa publicadora del contenido | [Ubisoft,Dovetail Games - Trains,Degica] |
| genres | Genero del contenido | [Action, Adventure, Racing, Simulation, Strategy] |
| app_name | Nombre del contenido | [Warzone, Soundtrack, Puzzle Blocks] |
| title | Titulo del contenido | [The Dream Machine: Chapter 4 , Fate/EXTELLA - Sweet Room Dream, Fate/EXTELLA - Charming Bunny] |
| url | URL de publicación del contenido | http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/ |
| release_date | Fecha de lanzamiento | [2018-01-04] |
| tags | etiquetas de contenido | [Simulation, Indie, Action, Adventure, Funny, Open World, First-Person, Sandbox, Free to Play] |
| discount_price | precio de descuento | [22.66, 0.49, 0.69] |
| reviews_url | Reviews de contenido | http://steamcommunity.com/app/681550/reviews/?browsefilter=mostrecent&p=1 |
| specs | Especificaciones | [Multi-player, Co-op, Cross-Platform Multiplayer, Downloadable Content] |
| price | Precio del contenido | [4.99, 9.99, Free to Use, Free to Play] |
| early_access | acceso temprano | [False, True] |
| id | identificador unico de contenido | [761140, 643980, 670290] |
| developer | Desarrollador | [Kotoshiro, Secret Level SRL, Poolians.com] |
| sentiment | Análisis de sentimiento | [Mixed, Very Positive, Positive, 3 user reviews] |
| metascore | Score por metacritic | [80, 74, 77, 75] |



Import relevant libraries.

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

## Read Data

I tried to open the json file with pd.read_json but i could not do it because of its format.

In [46]:
# pd.read_json()

That's why I decided to use "open" and then transform each row into a dictionary.

In [47]:
# Open file.
file = open('datasets/steam_games.json')

allNewLines = []

# Go through each line 
for line in file:
    # Create a dictionary of each line.
    myNewdict = ast.literal_eval(line)
    # Append to a list of all the lines in the file.
    allNewLines.append(myNewdict)

With the list of all the rows I can create a DataFrame.

In [48]:
df = pd.DataFrame(allNewLines)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


## Basic analisys

Let's check info, column names, types, duplicates

In [49]:
df.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'discount_price', 'reviews_url', 'specs', 'price',
       'early_access', 'id', 'developer', 'sentiment', 'metascore'],
      dtype='object')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [83]:
# Games with duplicate ids
df[df.duplicated(keep=False, subset='id')].sort_values(by='id')


,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games,Mostly Positive,86
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games,Mostly Positive,86
74,NaN,NaN,NaN,NaN,NaN,NaN,14.99,NaN,19.99,False,NaN,NaN,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)",Overwhelmingly Positive,91


In [82]:
df.drop_duplicates(inplace=True)

TypeError: unhashable type: 'list'

**CHANGE SOME DTYPES?????????????**

## Analysis of each column

### url & reviews_url
We do not need any url for the analysis so we will drop them.

In [51]:
df.drop(columns=['url', 'reviews_url'], inplace=True)

In [52]:
df.head()

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.83,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,NaN,"[Action, Indie, Casual, Sports]",1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [59]:
def cat_column_describe(name):
    """
    Basic description of a categorical column.
    """
    print('Describe:')
    print(df[name].describe())
    print('')
    print('Unique Values:')
    print(df[name].unique())
    print('')
    print('NaN amount:', df[name].isna().sum())

### publisher

In [60]:
cat_column_describe('publisher')

Describe:
count       24083
unique       8239
top       Ubisoft
freq          385
Name: publisher, dtype: object

Unique Values:
['Kotoshiro' 'Making Fun, Inc.' 'Poolians.com' ... 'OrtiGames/OrtiSoft'
 'INGAME' 'Bidoniera Games']

NaN amount: 8052


### genres

**What do I do with the list of this items????**

In [61]:
cat_column_describe('genres')

Describe:
count        28852
unique         883
top       [Action]
freq          1880
Name: genres, dtype: object

Unique Values:


TypeError: unhashable type: 'list'

In [62]:
cat_column_describe('app_name')

Describe:
count          32133
unique         32094
top       Soundtrack
freq               3
Name: app_name, dtype: object

Unique Values:
['Lost Summoner Kitty' 'Ironbound' 'Real Pool 3D - Poolians' ...
 'Russian Roads' 'EXIT 2 - Directions' 'Maze Run VR']

NaN amount: 2


In [63]:
cat_column_describe('title')

Describe:
count          30085
unique         30054
top       Soundtrack
freq               3
Name: title, dtype: object

Unique Values:
['Lost Summoner Kitty' 'Ironbound' 'Real Pool 3D - Poolians' ...
 'LOGistICAL: South Africa' 'Russian Roads' 'EXIT 2 - Directions']

NaN amount: 2050


In [ ]:
cat_column_describe('release_date')

Describe:
count          30068
unique          3582
top       2012-10-16
freq             100
Name: release_date, dtype: object

Unique Values:
['2018-01-04' '2017-07-24' '2017-12-07' ... '2016-11-19' 'January 2018'
 '2018-10-01']

NaN amount: 2067


genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'discount_price', 'reviews_url', 'specs', 'price',
       'early_access', 'id', 'developer', 'sentiment', 'metascore'],

In [65]:
cat_column_describe('tags')

Describe:
count                    31972
unique                   15395
top       [Casual, Simulation]
freq                      1292
Name: tags, dtype: object

Unique Values:


TypeError: unhashable type: 'list'

---
Otras cosas para borrar luego

In [ ]:
def num_column_describe(name, allUnique = False):
    print('Describe:')
    print(df[name].describe())
    if (allUnique):
        print('')
        print('Unique Values:')
        print(df[name].unique())
    print('')
    print('NaN amount:', df[name].isna().sum())
    print('')
    print('Unique Values:')
    print('Sorted lower:', np.sort(df[name].unique())[0:10])
    print('Sorted higher:', np.sort(df[name].unique())[-10:])

In [53]:
df.describe()

,discount_price
count,225.000000
mean,11.930533
std,17.492643
min,0.490000
25%,1.390000
50%,4.190000
75%,22.660000
max,139.990000
